In [1]:
import folium
from folium.plugins import TimestampedGeoJson
import data_api as da

In [2]:
bus_ids = [51, 50, 31, 32, 4, 38, 30, 26, 13, 49, 40, 6, 29, 20, 10, 41, 3, 58, 7]

In [3]:
routes_gj = da.get_routes_geojson(bus_ids)
grid_gj = da.get_grid_geojson(bus_ids, ('weekday', '00:00', '23:59'), stations_source='here')

In [4]:
m = folium.Map(location=[55.6795535, 12.542231], zoom_start=13, tiles='OpenStreetMap')
routes = folium.GeoJson(routes_gj, name='routes', style_function=lambda feature: {
    'color': 'blue'
})

grid = TimestampedGeoJson(
    grid_gj, 
    period='PT1H',
    duration='PT1M',
    date_options='HH:mm',
    auto_play=False,
    max_speed=1,
    loop=False,
    time_slider_drag_update=True,
    loop_button=True
)

routes.add_to(m)
grid.add_to(m)

In [5]:
folium.features.GeoJsonTooltip(fields=['bus_id', 'name'], aliases=['id', 'bus']).add_to(routes)
folium.LayerControl().add_to(m)

In [6]:
m